# Wikipedia爬蟲   
## 定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os

## 將搜尋的關鍵字，變成UTF-8編碼
https://zh.wikipedia.org/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2  
這是關鍵字'網路爬蟲'的wiki網址，後面那一段是'網路爬蟲'的UTF-8編碼

In [1]:
def keyword_to_UTF8(keyword = None):
    utf8_url = repr(keyword.encode('UTF-8')).upper() #B'\XE7\XB6\XB2\XE8\XB7\XAF\XE7\X88\XAC\XE8\X9F\XB2'
    utf8_url = utf8_url.replace('\\X', '%') # B'%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2'
    root_keyword_link = '/wiki/' + utf8_url[2:-1]
    return root_keyword_link

In [10]:
def wiki():
    keyword = str(input('輸入關鍵字： '))
    
    # 模擬封包
    headers = {
        'authority': 'zh.wikipedia.org',
        'method': 'GET',
        'path': keyword_to_UTF8(keyword = keyword),
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'cache-control': 'max-age=0',
        'cookie': 'TBLkisOn=0; VEE=wikitext; WMF-Last-Access=18-Dec-2019; WMF-Last-Access-Global=18-Dec-2019; GeoIP=TW:TPE:Taipei:25.05:121.53:v4',
        #'if-modified-since': 'Mon, 16 Dec 2019 05:20:12 GMT',
        'referer': 'https://zh.wikipedia.org/zh-tw/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'
    }
    url = 'https://zh.wikipedia.org' + keyword_to_UTF8(keyword = keyword)
    resp = requests.get(url, headers=headers)
    resp.encoding = 'utf-8'
    html = BeautifulSoup(resp.text, "lxml")

    # 讀取文字部分，並從在檔案 WikiArticle裡頭
    
    context = html.find('div', attrs = {'id': 'mw-content-text'}).find_all('p')
    
    output_dir = 'WikiArticle'
    output_file = '{outputdir}_{outputfile}.txt'.format(outputdir = output_dir, outputfile = keyword)

    with open(output_file, "w", encoding="utf-8") as fh:
            for paragraph in context:
                f = fh.write(paragraph.get_text())
    
    # 查詢文中外部連結關鍵字，並存成dict
        
    keyword_go_on = str(input('是否查詢文中出現的外部連結關鍵字(y/n):'))
    external_link_dict = {}
    if keyword_go_on == 'y':
        for ext_link in context:
            a_tag = ext_link.find_all('a', href = re.compile('^/wiki/'))
            if len(a_tag) > 0:
                for link in a_tag:
                    a_link = 'https://zh.wikipedia.org' + link['href']
                    a_keyword = link.text
                    print('外部連結：[%s] %s' %(a_keyword, a_link))
                    external_link_dict[a_keyword] = a_link
    elif keyword_go_on == 'n':
        print('Thank you')
    
    # 遞迴搜尋
    
    article_go_on = str(input('是否繼續搜尋(y/n)：'))
    if article_go_on == 'y':
        wiki()
    elif article_go_on == 'n':
        print('Thank you')
        
                    
            

In [11]:
wiki()

輸入關鍵字： 網路爬蟲
是否查詢文中出現的外部連結關鍵字(y/n):y
外部連結：[全球資訊網] https://zh.wikipedia.org/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結：[網路機器人] https://zh.wikipedia.org/wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA
外部連結：[網路] https://zh.wikipedia.org/wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結：[搜尋引擎] https://zh.wikipedia.org/wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結：[robots.txt] https://zh.wikipedia.org/wiki/Robots.txt
外部連結：[網站] https://zh.wikipedia.org/wiki/%E7%BD%91%E7%AB%99
外部連結：[超連結] https://zh.wikipedia.org/wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結：[HTML] https://zh.wikipedia.org/wiki/HTML
外部連結：[網頁] https://zh.wikipedia.org/wiki/%E7%B6%B2%E9%A0%81
外部連結：[網際網路] https://zh.wikipedia.org/wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結：[伺服器] https://zh.wikipedia.org/wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結：[超文字傳輸協定] https://zh.wikipedia.org/wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A
是否繼續搜尋(y/n)：y
輸入關鍵字： 網路機器人
是否查詢文中出現的外部連結關鍵字(y/n):y
外部連結：[網際網路] https://zh.wikipedia.org/w

In [5]:
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
#print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
#print(root_keyword_link)
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#

#for paragraph in content:
    #print(paragraph.get_text())


In [32]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href = re.compile('^/wiki/'))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string['href']
            a_keyword = link_string.text
            print('外部連結 [%s] %s' %(a_keyword, a_link))

外部連結 [全球資訊網] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結 [網路機器人] /wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA
外部連結 [網路] /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結 [搜尋引擎] /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結 [robots.txt] /wiki/Robots.txt
外部連結 [網站] /wiki/%E7%BD%91%E7%AB%99
外部連結 [超連結] /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結 [HTML] /wiki/HTML
外部連結 [網頁] /wiki/%E7%B6%B2%E9%A0%81
外部連結 [網際網路] /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結 [伺服器] /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結 [超文字傳輸協定] /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A
